## Elasticsearch: The Definitive Guide - Python

Following the examples in the book, here are Python snippets that achieve the same effect.

Documentation for the Python libs:

Low-level API:

https://elasticsearch-py.readthedocs.io/en/master/index.html

Expressive DSL API (more "Pythonic")

http://elasticsearch-dsl.readthedocs.io/en/latest/index.html

Github repo for DSL API:

https://github.com/elastic/elasticsearch-dsl-py


In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
import index
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from pprint import pprint

es = Elasticsearch(
    'localhost',
    # sniff before doing anything
    sniff_on_start=True,
    # refresh nodes after a node fails to respond
    sniff_on_connection_fail=True,
    # and also every 60 seconds
    sniffer_timeout=60
)

r = index.populate()
print('{} items created'.format(len(r['items'])))

# Let's repopulate the index as we deleted 'gb' in earlier chapters:
# Run the script: populate.ipynb

14 items created


### Reducing Words to Their Root Form

Most languages of the world are inflected, meaning that words can change their form to express differences in the following:

* Number: fox, foxes
* Tense: pay, paid, paying
* Gender: waiter, waitress
* Person: hear, hears
* Case: I, me, my
* Aspect: ate, eaten
* Mood: so be it, were it so

While inflection aids expressivity, it interferes with retrievability, as a single root word sense (or meaning) may be represented by many different sequences of letters

Stemming attempts to remove the differences between inflected forms of a word, in order to reduce each word to its root form. For instance foxes may be reduced to the root fox.

If stemming were easy, there would be only one implementation. Unfortunately, stemming is an inexact science that suffers from two issues: understemming and overstemming.

**The root form of a word may not even be a real word**. The words ```jumping``` and ```jumpiness``` may both be stemmed to ```jumpi```. It doesn’t matter—as long as the same terms are produced at index time and at search time, search will just work.

Understemming is the failure to reduce words with the same meaning to the same root. For example, ```jumped``` and ```jumps``` may be reduced to ```jump```, while ```jumping``` may be reduced to ```jumpi```. **Understemming reduces retrieval**; relevant documents are not returned.

Overstemming is the failure to keep two words with distinct meanings separate. For instance, ```general``` and ```generate``` may both be stemmed to ```gener```. **Overstemming reduces precision**: irrelevant documents are returned when they shouldn’t be.

#### Lemmatization ####

A lemma is the canonical, or dictionary, form of a set of related words—the lemma of paying, paid, and pays is pay. Sometimes the morphology differs: is, was, am, and being is be.

Lemmatization, like stemming, tries to group related words, but it goes one step further than stemming in that it tries to group words by their word sense, or meaning. The same word may represent two meanings—for example,wake can mean to wake up or a funeral. While lemmatization would try to distinguish these two word senses, stemming would incorrectly conflate them.

Lemmatization is a much more complicated and expensive process that needs to understand the context in which words appear in order to make decisions about what they mean. In practice, stemming appears to be just as effective as lemmatization, but with a much lower cost.

In [3]:
isinstance(1,int)

True